In [32]:
#Import and install the necessary packages
import requests
import pandas as p
import numpy as np
import time as t

In [33]:
df_dict = p.read_excel(r'L:\PRIV\ToxValDB\Document Mapping\1- In Progress\source_hawc\hawc_original_12_06_21.xlsx',sheet_name = None)
ccte_map = p.read_excel(r'L:\PRIV\ToxValDB\Document Mapping\toxval_document_map_ccte.xlsx')
icf_map = p.read_excel(r'L:\PRIV\ToxValDB\Document Mapping\toxval_document_map_icf.xlsx')

In [34]:
#do document matching on each sheet
for i in range(len(df_dict)):
    df = df_dict.get('sheet'+str(i)) #load in the i'th sheet in big hawc
    for j in range (len(df)):
    #replace clowder_id and doc name with empty space. This gives us something to look for when finding docs that need matching
        
        df.loc[j,'clowder_id'] = '' 
        df.loc[j,'document_name'] = ''

#these arrays will hold the long refs for documents we have matched. Now instead of looping through doc map for
#records with repeat titles, we just look back at these arrays. Drastically improves runtime
      
        used_ref = []
        used_title = []
        used_clowder_id =[]
        used_doc_name = []
        

#Look for long refs and match them to long refs in icf map. Then replace clowder ids and document names with those in clowder
    for j in range (len(df)):
        record = str(df.loc[j,'animal_group.experiment.study.full_citation']).lower()
        if record in used_ref:
            idx = used_ref.index(record)
            df.loc[j,'clowder_id'] = used_clowder_id[idx]
            df.loc[j,'document_name'] = used_doc_name[idx]
        if df.loc[j,'clowder_id'] == '':
            for k in range (len(icf_map)):
                doc = str(icf_map.loc[k, 'long_ref']).lower()
                if record in doc:
                    used_ref.append(record)
                    used_title.append(df.loc[j,'animal_group.experiment.study.title'])
                    used_clowder_id.append(icf_map.loc[k,'clowder_id'])
                    used_doc_name.append(icf_map.loc[k,'document_name'])
                    df.loc[j,'clowder_id'] = icf_map.loc[k,'clowder_id']
                    df.loc[j,'document_name'] = icf_map.loc[k,'document_name']

In [36]:
#Maybe the long refs vary slightly or there was trouble reading accent marks on certain names.
#Lets use the "in" operator to look for titles in the long refs instead. I found that this picks up a few straggler docs
#in each sheet (which amounts to hundreds of records. Pretty good thing for programs instead of by hand inspection)

#Again, we load in each sheet individually and perform the same process as above, this time we look at title in long ref
#instead of long ref in long ref

for i in range(len(df_dict)):
    df = df_dict.get('sheet'+str(i))
    for j in range (len(df)):
        if df.loc[j,'clowder_id'] == '':
            record = str(df.loc[j,'animal_group.experiment.study.title']).lower()
            if record in used_title:
                idx = used_title.index(record)
                df.loc[j,'clowder_id'] = used_clowder_id[idx]
                df.loc[j,'document_name'] = used_doc_name[idx]
        for k in range (len(icf_map)):
            doc = str(icf_map.loc[k, 'long_ref']).lower()
            if record in doc:
                used_title.append(record)
                used_clowder_id.append(icf_map.loc[k,'clowder_id'])
                used_doc_name.append(icf_map.loc[k,'document_name'])
                df.loc[i,'clowder_id'] = icf_map.loc[k,'clowder_id']
                df.loc[i,'document_name'] = icf_map.loc[k,'document_name']


In [42]:
length = 0
counter = 0
for i in range (len(df_dict)):
    df = df_dict.get('sheet'+str(i))
    length = length + len(df)
    for j in range(len(df)):
        if df.loc[j,'clowder_id'] !='':
            counter = counter + 1
print(counter)
print(length)
print((counter/length)*100, 'percent of the records have been matched programatically')

3590
5399
66.49379514724949 percent of the records have been matched programatically


In [43]:
#Check validity of clowder ids

api_key = '689d94c2-1a11-4940-b2eb-a0faff85c6be'
used_id = []
validity = []
for i in range (len(df_dict)):
    df = df_dict.get('sheet'+str(i))
    for j in range (len(df)):
        if df.loc[j,'clowder_id'] != '':
#Many recoreds come from the same document. Save time by first seeing if we already checked the validity of the clowder id
            clowder_id = df.loc[j,'clowder_id']
            if clowder_id not in used_id:
                t.sleep(1) #Courtesy sleep time
#Use requests to interact with clowder api. Save the response code and clowder id in arrays.
                response = requests.get('https://clowder.edap-cluster.com/api/files/'+clowder_id, headers = {'X-API-Key': api_key})
                used_id.append(clowder_id)
                validity.append(response)

In [44]:
print(len(validity))

164


In [45]:
print(validity)

[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response

In [48]:
#Check the validity of each clowder id aquired to make sure it is attached to a clowder document
for i in range (len(df_dict)):
    df = df_dict.get('sheet'+str(i))
    #Create the dataframe column since it wasn't there before
    val_arr = np.zeros(len(df))
    df['clowder_validity'] = val_arr
    for j in range(len(df)):
        if df.loc[j,'clowder_id'] in used_id:
#Consult the arrays we made before out of clowder ids and their corresponding response codes
            idx = used_id.index(df.loc[j,'clowder_id'])
            val = validity[idx]
#True if clowder id gives a 200 response code, false if it doesnt't
            if val.status_code == 200:
                df.loc[j,'clowder_validity'] = 'TRUE'
            else:
                df.loc[j,'clowder_validity'] = 'FALSE'
#Note: Not all records were matched to documents and therefore do not have clowder ids. These will remain as 0s

In [52]:
#Count the number of matched records
counter2 = 0
for i in range(len(df_dict)):
    df = df_dict.get('sheet'+str(i))
    for j in range(len(df)):
        if df.loc[j,'clowder_validity'] != 0:
            counter2 = counter2 + 1
print(counter2)

3590


In [57]:
#Save the dataframe to an excel spreadsheet
writer = p.ExcelWriter('big_hawc_match.xlsx', engine='xlsxwriter')
for i in range (len(df_dict)):
    df = df_dict.get('sheet'+str(i))
    df.to_excel(writer, sheet_name = 'sheet'+str(i), index = False)
writer.save()